In [1]:
!git clone "https://github.com/srajan-kiyotaka/Pothole-Detection.git" 

fatal: destination path 'Pothole-Detection' already exists and is not an empty directory.


In [11]:
# Importing the csv file

import pandas as pd

features = pd.read_csv(r"/content/Pothole-Detection/Dataset_Info.csv")

features.head(10)

,Image ID,Pothole,Number of Potholes,Level,Unnamed: 4
0,pothole_image_1299,1,3,B,NaN
1,pothole_image_10064,1,1,A,NaN
2,pothole_image_10079,1,1,A,NaN
3,pothole_image_10084,1,1,A,NaN
4,pothole_image_10087,1,2,A,NaN
5,pothole_image_10110,1,1,A,NaN
6,pothole_image_10119,1,1,A,NaN
7,pothole_image_10120,1,1,A,NaN
8,pothole_image_10135,1,1,A,NaN
9,pothole_image_10139,1,1,A,NaN


In [12]:
features.isna().sum()

Image ID                  0
Pothole                   0
Number of Potholes        0
Level                     0
Unnamed: 4            30000
dtype: int64

In [13]:
features = features.drop('Unnamed: 4', axis = 1)

In [14]:
features.head(10)

,Image ID,Pothole,Number of Potholes,Level
0,pothole_image_1299,1,3,B
1,pothole_image_10064,1,1,A
2,pothole_image_10079,1,1,A
3,pothole_image_10084,1,1,A
4,pothole_image_10087,1,2,A
5,pothole_image_10110,1,1,A
6,pothole_image_10119,1,1,A
7,pothole_image_10120,1,1,A
8,pothole_image_10135,1,1,A
9,pothole_image_10139,1,1,A


In [15]:
features.isna().sum()

Image ID              0
Pothole               0
Number of Potholes    0
Level                 0
dtype: int64

In [16]:
all_image_names = features['Image ID']
features = features.drop('Image ID', axis = 1)

pothole_or_not = features['Pothole']
pothole_info = features['Number of Potholes']
pothole_level = features['Level']

pothole_or_not = pd.get_dummies(pothole_or_not, columns = ['Pothole'])
pothole_or_not.columns = ['Normal Road', 'Pothole']
pothole_level = pd.get_dummies(pothole_level, columns = ['Level'])

features = pothole_or_not.join(pothole_info)
features = features.join(pothole_level)

print(features.head(10))

   Normal Road  Pothole  Number of Potholes  A  B  C  S
0            0        1                   3  0  1  0  0
1            0        1                   1  1  0  0  0
2            0        1                   1  1  0  0  0
3            0        1                   1  1  0  0  0
4            0        1                   2  1  0  0  0
5            0        1                   1  1  0  0  0
6            0        1                   1  1  0  0  0
7            0        1                   1  1  0  0  0
8            0        1                   1  1  0  0  0
9            0        1                   1  1  0  0  0


In [17]:
from sklearn.utils import shuffle

image_names_shuffled, labels_shuffled = shuffle(all_image_names, features)

In [18]:
from sklearn.model_selection import train_test_split

X_train_image_names, X_test_image_names, y_train, y_test = train_test_split(
    image_names_shuffled, labels_shuffled, test_size=0.3, random_state=1)

In [20]:
train_pothole_or_not = y_train[['Normal Road', 'Pothole']]
train_pothole_info = y_train['Number of Potholes']
train_pothole_level = y_train[['A', 'B', 'C', 'S']]

test_pothole_or_not = y_test[['Normal Road', 'Pothole']]
test_pothole_info = y_test['Number of Potholes']
test_pothole_level = y_test[['A', 'B', 'C', 'S']]

In [31]:
# train_pothole_or_not.head()
# train_pothole_info.head()
# train_pothole_level.head()

# test_pothole_or_not.head()
# test_pothole_info.head()
# test_pothole_level.head()

In [34]:
import cv2

def get_image(file_location):
    img = cv2.imread(file_location)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = img/255
    img = cv2.resize(img,(256, 256))
    return img

In [35]:
import keras
import numpy as np

class My_Custom_Generator(keras.utils.all_utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size = 256) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            get_image(f'/content/Pothole-Detection/Unified Dataset/{str(file_name)}.jpg')
               for file_name in batch_x]), np.array(batch_y)

In [37]:
batch_size = 300

my_training_batch_generator = My_Custom_Generator(X_train_image_names, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test_image_names, y_test, batch_size)